In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


/Users/aa406165/Git/qiskit-core/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [5]:
from qiskit.providers.aer import AerSimulator
backend = AerSimulator()

In [6]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shots=3000)

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=(1.8166666666666667+0j), variance=(11.129208444444444+0j), confidence_interval=[((1.723967817740224+0j), (1.9093655155931093+0j))])

In [7]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=(1.787109375+0j), variance=(11.234569549560547+0j), confidence_interval=[((1.6271020885725256+0j), (1.9471166614274744+0j))])

In [8]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for this backend: aer_simulator


CompositeResult(items=[ExpectationValueResult(value=(1.7813333333333332+0j), variance=(11.257481777777778+0j), confidence_interval=[((1.6875398363989027+0j), (1.8751268302677637+0j))]), ExpectationValueResult(value=(0.1966666666666672+0j), variance=(10.59664488888889+0j), confidence_interval=[((0.10268679182323055+0j), (0.2906465415101039+0j))])])

### Exact simulation by SaveExpectationValueVariance

In [9]:
from qiskit.evaluators import ExactExpectationValue

In [10]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
%time expval.evaluate([0, 1, 1, 2, 3, 5])

CPU times: user 9.58 ms, sys: 629 µs, total: 10.2 ms
Wall time: 10.1 ms


ExpectationValueResult(value=1.8420921273153708, variance=6.432763522806668, confidence_interval=None)

In [11]:
%time expval.evaluate([1, 1, 2, 3, 5, 8])

CPU times: user 2.55 ms, sys: 329 µs, total: 2.88 ms
Wall time: 2.83 ms


ExpectationValueResult(value=0.15611896641425527, variance=5.356814314603938, confidence_interval=None)

### Transpiled Circuits

In [12]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
# expval.set_transpile_options(initial_layout=[1, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
               ┌────┐┌──────────────┐┌────┐┌────────┐     ┌────┐»
      q_0 -> 0 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤┌─┴─┐├────┤»
      q_1 -> 1 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘└───┘└────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 3 ─────────────────────────────────────────────────»
                                                                »
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c9: 2/═════════════════════════════════════════════════»
                                                                »
«               ┌──────────────┐┌────┐┌────────┐     ┌────┐┌──────────────┐»
«      q_0 -> 0 ┤ Rz(θ[2] + π) ├┤ √X ├┤ Rz(3π) ├─

### Transpile options and Run options

In [13]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=(1.5220920036035928+0j), variance=(12.004484853121927+0j), confidence_interval=[((1.5166702473781495+0j), (1.527513759829036+0j))])

In [14]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=(1.512939453125+0j), variance=(12.053604543209076+0j), confidence_interval=[((1.4528491556493117+0j), (1.5730297506006883+0j))])

In [15]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=(1.9933333333333336+0j), variance=(11.589288888888888+0j), confidence_interval=[((1.684784068272271+0j), (2.3018825983943962+0j))])

### Composite Evaluator

In [16]:
from qiskit.evaluators import JointEvaluator

In [17]:
len(expval.transpiled_circuits)

3

In [18]:
joint_evaluator = JointEvaluator([expval, expval, expval])

In [19]:
len(joint_evaluator.transpiled_circuits) # 3 × 3

9

In [20]:
joint_evaluator.evaluate([0, 1, 1, 2, 3, 5])

CompositeResult(items=[ExpectationValueResult(value=(1.575927734375+0j), variance=(11.97433751821518+0j), confidence_interval=[((1.5160921290116833+0j), (1.6357633397383167+0j))]), ExpectationValueResult(value=(1.498046875+0j), variance=(12.013070464134216+0j), confidence_interval=[((1.4381256023679945+0j), (1.5579681476320055+0j))]), ExpectationValueResult(value=(1.4755859375+0j), variance=(12.064022183418274+0j), confidence_interval=[((1.4154714573640874+0j), (1.5357004176359126+0j))])])

In [21]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=(1.58447265625+0j), variance=(11.9540536403656+0j), confidence_interval=[((1.524723186269698+0j), (1.644222126230302+0j))]), ExpectationValueResult(value=(0.25146484375+0j), variance=(11.723543882369995+0j), confidence_interval=[((0.19143538427931606+0j), (0.31149430322068394+0j))]), ExpectationValueResult(value=(0.86572265625+0j), variance=(12.33379876613617+0j), confidence_interval=[((0.8044507659676063+0j), (0.9269945465323937+0j))])])

### Mitigator

In [22]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(backend, mitigation="tensored", refresh=1, shots=2000, mit_pattern=[[0], [1]])
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")
print(f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")

w/o mitigation shots=2000, result=ExpectationValueResult(value=(1.594+0j), variance=(11.756284+0j), confidence_interval=[((1.4747452166165294+0j), (1.7132547833834708+0j))])
w/  mitigation shots=2000, result=ExpectationValueResult(value=(1.675400798068432+0j), variance=(11.59415466283846+0j), confidence_interval=[((1.5575238507133178+0j), (1.7932777454235462+0j))])
